### Importing Libraries
The first step in any Python script or Jupyter notebook is to import the necessary libraries. Here, we are importing the libraries we need to optimize our portfolios and run the backtest with Zipline Reloaded.

In [ ]:
import os
import pandas as pd
import numpy as np
import riskfolio as rp

from zipline import run_algorithm
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume, VWAP, AnnualizedVolatility, SimpleBeta
from zipline.api import (
    symbol,
    attach_pipeline,
    calendars,
    pipeline_output,
    date_rules,
    time_rules,
    get_datetime,
    record,
    order_target_percent,
    get_open_orders,
    schedule_function,
    set_commission,
    set_slippage,
    set_benchmark
)
from zipline.finance import commission, slippage
from zipline.data import bundles
from zipline.utils.run_algo import load_extensions

import warnings
warnings.filterwarnings('ignore')

### Option 1: Use the built in bundle with free data
This option uses the built in data bundle provided by Zipline. It then acquires free US equities data that extend through 2018.

In [ ]:
os.environ["QUANDL_API_KEY"] = "YOUR_API_KEY"
bundle = "quandl"
bundles.ingest(bundle)

### Option 2: Use the custom bundle with premium data
This option uses the custom bundle with premium data. You can find the detailed instructions to set this up here: https://www.pyquantnews.com/free-python-resources/how-to-ingest-premium-market-data-with-zipline-reloaded

In [ ]:
os.environ["DATALINK_API_KEY"] = "YOUR_API_KEY"
bundle = "quotemedia"
load_extensions(True, [], False, os.environ)
bundles.ingest(bundle)

### Trade settings
Instead of hard coding these settings inside the backtest, we'll make it easier on ourselves by setting them up front.

This code defines `leverage` as 1.5, indicating that the trading strategy can borrow up to 50% more capital than the amount initially invested. `window_length` and `bar_count` are both set to 126 (21 days multiplied by 6, representing six trading months), defining the number of trading bars (e.g., days) to consider for calculations. `top_n` is set to 100, specifying that the strategy will focus on the top 100 assets.

In [ ]:
leverage = 1.5
window_length = 21 * 6
bar_count = window_length
top_n = 100

### Create the Pipeline
A Pipeline is a framework that allows for the definition and efficient computation of a wide array of financial data over a set of assets.

Inside the Pipeline, we use factors like `AverageDollarVolume`, `VWAP` (volume-weighted average price), `AnnualizedVolatility`, and `SimpleBeta` are calculated, each using the window_length to define the lookback period. The `SimpleBeta` factor specifically calculates the beta of stocks relative to the S&P 500 (SPY). A screening condition is set up to filter stocks where the `VWAP` is above $50, annualized volatility is between 35-60%, and beta is between 0.5 and 1.0. The pipeline returns a dictionary of columns where longs are stocks in the top top_n by dollar volume, filtered by the defined conditions.

In [ ]:
def make_pipeline():
    
    dollar_volume = AverageDollarVolume(window_length=window_length)
    vwap = VWAP(window_length=window_length)
    annualized_volatility = AnnualizedVolatility(window_length=window_length)
    beta = SimpleBeta(target=symbol("SPY"), regression_length=window_length)

    screen = (
        (vwap > 50.0) & 
        (0.35 < annualized_volatility <= 0.60) & 
        (0.5 < beta < 1.0)
    )
    
    return Pipeline(
        columns={
            "longs": dollar_volume.top(top_n),
        },
        screen=screen
    )

### Recalculate the pipeline

The function `before_trading_start` executes before the rabalance period. It retrieves the output of a previously defined pipeline named "screener" and stores it in the context object under the attribute screener. This lets use use it throughout other parts of the trading algorithm.

In [ ]:
def before_trading_start(context, data):
    context.screener = pipeline_output("screener")

### Initialize the backtest

The initialize function sets up the initial configuration for our backtest.

It attaches a defined pipeline named "screener" to the trading context and schedules a function rebalance to run at the start of each week when the market opens, specifically for the US equities calendar. Commission and slippage models are defined to manage transaction costs and market impact, respectively, with specific parameters for per-share costs and volume limits. Additionally, the S&P 500 ETF (SPY) is set as the benchmark for performance comparison.

In [ ]:
def initialize(context):
    attach_pipeline(make_pipeline(), "screener")
    
    schedule_function(
        rebalance,
        date_rules.week_start(),
        time_rules.market_open(),
        calendar=calendars.US_EQUITIES,
    )

    # Set up the commission model to charge us per share and a volume slippage model
    set_commission(
        us_equities=commission.PerShare(
            cost=0.005,
            min_trade_cost=2.0
        )
    )
    set_slippage(
        us_equities=slippage.VolumeShareSlippage(
            volume_limit=0.0025, 
            price_impact=0.01
        )
    )
    # SPY is only available in the premium data bundle. You can use any symbol you
    # want for the benchmark.
    set_benchmark(symbol("SPY"))

### Create a function to compute the weights

The `compute_weights` function calculates optimal portfolio weights based on historical price data of assets.

The function initializes a Riskfolio `portfolio` object with the asset returns and specifies the methods for calculating mean returns and covariance, using historical data. Note the covariance matrix is calculated using the Ledoit-Wolf covariance shrinkage method. It sets a minimum expected return for the optimization based on a specified value. Finally, the function optimizes the portfolio using a mean-variance model, considering historical data, and returns the optimized weights.

In [ ]:
def compute_weights(returns):

    port = rp.Portfolio(returns=returns)
    port.assets_stats(method_mu="hist", method_cov="ledoit")
    port.lowerret = 0.00085
    
    return port.rp_optimization(
        model="Classic", 
        rm="MV", 
        b=None
    )

### Set our rebalance logic
In the rebalance function, we extract the asset universe from our pipeline's DataFrame and identify which stocks to go long on and which to divest. We calculate historical prices for the target assets, and if the returns are valid, we compute optimal weights using a volatility targeting algorithm, adjusting for leverage. On rebalance days, we log our actions, showing the number of assets longed, divested, and the portfolio's value. Finally, we execute trades to realign our portfolio according to these calculated weights, ensuring it meets our strategic objectives.

In [ ]:
def rebalance(context, data):

    # Get the output of the Pipeline which is a pandas DataFrame.
    screened = context.screener

    # Get the list the universe of assets to consider in portfolio
    # construction
    assets = screened.index

    # Get the symbols of those stocks we want to go long
    longs = assets[screened.longs]

    # Figure out which assets are no longer required in our portfolio
    # based on universe construction. Build a DataFrame suitable for
    # passing into exec_trades
    divest = list(set(context.portfolio.positions.keys()) - set(longs))
    divest_weights = pd.DataFrame(
        np.zeros(len(divest)), 
        index=divest, 
        columns=["weights"]
    )

    print(
        f"{get_datetime().date()} "
        f"Longs={len(longs)} "
        f"Divest={len(divest)} "
        f"Value:={context.portfolio.portfolio_value} "
    )

    # Get the historic prices for the assets we want to buy.
    prices = data.history(
        longs, 
        "price", 
        bar_count=bar_count, 
        frequency="1d"
    )

    # Divest the assets we no longer want
    exec_trades(data, assets=divest, weights=divest_weights)

    # Compute the returns of the assets we want to buy, drop any symbols
    # where there are NaNs (no data), and select assets where we have daily
    # returns that change ie avoid stocks where the price is the the same
    # every day
    returns = prices.pct_change()[1:]
    returns.dropna(how="any", axis=1, inplace=True)
    returns = returns.loc[:, (returns != 0).any(axis=0)]

    # Make sure we have asset returns to optimize against
    if returns.empty:
        return

    # Get the weights based on our constrained volatility targeting algorithm.
    weights = compute_weights(returns)

    # If we have weights, execute the trades.
    if weights is not None:

        weights *= leverage
        
        exec_trades(data, assets=longs, weights=weights)

### Execute the trades
We loop through every asset, determine if it's tradeable, the asset is weighted, and there are no open orders. Then we order the target percent and Zipline rebalances our portfolio for us.

In [ ]:
def exec_trades(data, assets, weights):
    # Loop through every asset...
    for asset in assets:
        # ...if the asset is tradeable and there are no open orders...
        if data.can_trade(asset) and asset in weights.index and not get_open_orders(asset):
            # ...execute the order against the target percent
            target_percent = weights.at[asset, "weights"]
            order_target_percent(asset, target_percent)

### Analyze the results
We can ask Zipline to run a function at the conclusion of the backtest. In this case, we simply print the equity curve and cache the backtest output.

In [ ]:
def analyze(context, perf):
    # You can execute any arbitrary code here. This includes saving results
    # to a database, sending yourself an email, send a note to Discord, or
    # even executing trades.
    perf.portfolio_value.plot()
    perf.to_pickle("constrained.pickle")

### Run the backtest
The `run_algorithm` funtion kicks off the backtest. It takes the `start` and `end` date, the `initialize` function that runs at the inception of the backtest, the `analyze` function that runs at the conclusion of the backtest, the `capital` to start the backtest with, and the name of the bundle.

In [ ]:
perf = run_algorithm(
    start=pd.Timestamp("2015-01-01"),
    end=pd.Timestamp("2023-12-31"),
    initialize=initialize,
    before_trading_start=before_trading_start,
    analyze=analyze,
    capital_base=100_000,
    bundle="quotemedia",
)

### Cache the results for later analysis
Since `perf` is a pandas DataFrame, we can use the `to_pickle` method to save a serialized copy of the pandas DataFrame to disk. We'll need this later when we assess this strategy's performance.

In [ ]:
perf.to_pickle("risk-parity.pickle")